In [26]:
#直接查詢(無法查詢)
import google.generativeai as genai
import os
from IPython.display import display,Markdown 

genai.configure(api_key=os.environ['GEMINI_API_KEY'])
model = genai.GenerativeModel(
    model_name='gemini-2.0-flash-exp'
)
response = model.generate_content(
   "今日歐元和美金的匯率?" 
)
display(Markdown(response.text))

目前歐元兌美元的匯率會不斷變動，建議您參考以下幾個可靠的資訊來源查詢：

* **Google 搜尋:** 直接在 Google 搜尋 "歐元 美元 匯率"，Google 會顯示即時匯率資訊，並附有圖表。
* **專業財經網站:** 例如 Bloomberg、Reuters、Yahoo Finance 等，這些網站提供更詳細的匯率資訊，包括歷史數據和圖表。
* **銀行網站或手機 App:** 您可以查詢您常用的銀行網站或手機 App，它們通常會提供最新的匯率資訊。
* **外匯交易平台:** 如果您有外匯交易需求，可以在交易平台上查詢即時匯率。

**注意事項：**

*   **匯率隨時都在變動：** 匯率是動態變化的，建議查詢時以最新資訊為準。
*   **不同來源的匯率可能略有差異：** 因為來源不同，例如銀行的買入價和賣出價會不同，以及不同資訊平台的數據更新頻率不同，匯率可能會有些微差異。
*   **交易匯率與牌告匯率不同：** 如果您要進行外匯交易，實際的交易匯率會與牌告匯率不同，通常銀行或外匯平台會收取一些手續費。

**總之，要查詢歐元兌美元的即時匯率，建議使用上述提到的可靠來源，並注意匯率的動態變化。**


In [27]:
#使用外部api
#api的使用說明:https://frankfurter.dev/
import requests
from pprint import pprint
url = "https://api.frankfurter.app/latest" #目前最新的
#url = "https://api.frankfurter.app/2024-03-01" #指定日期
#url = "https://api.frankfurter.app/1999-01-04?base=USD&symbols=EUR" #指定日期,base和symbols
response = requests.get(url)
pprint(response.json())

{'amount': 1.0,
 'base': 'EUR',
 'date': '2025-01-31',
 'rates': {'AUD': 1.6702,
           'BGN': 1.9558,
           'BRL': 6.0677,
           'CAD': 1.5035,
           'CHF': 0.9449,
           'CNY': 7.5363,
           'CZK': 25.166,
           'DKK': 7.4618,
           'GBP': 0.83608,
           'HKD': 8.099,
           'HUF': 407.95,
           'IDR': 16941,
           'ILS': 3.7199,
           'INR': 89.99,
           'ISK': 146.7,
           'JPY': 160.99,
           'KRW': 1506.62,
           'MXN': 21.477,
           'MYR': 4.6301,
           'NOK': 11.7373,
           'NZD': 1.8391,
           'PHP': 60.682,
           'PLN': 4.213,
           'RON': 4.9767,
           'SEK': 11.474,
           'SGD': 1.4091,
           'THB': 34.915,
           'TRY': 37.266,
           'USD': 1.0393,
           'ZAR': 19.3588}}


In [28]:
#使用外部api
#api的使用說明:https://frankfurter.dev/
import requests
from pprint import pprint
#url = "https://api.frankfurter.app/latest" #目前最新的
#url = "https://api.frankfurter.app/2024-03-01" #指定日期
symbol = 'EUR'
url = f"https://api.frankfurter.app/1999-01-04?base=USD&symbols={symbol}" #指定日期,base和symbols
response = requests.get(url)
pprint(response.json())
response.json()['rates'][symbol]

{'amount': 1.0, 'base': 'USD', 'date': '1999-01-04', 'rates': {'EUR': 0.84825}}


0.84825

In [60]:
#定義function calling

def get_exchange_rate(currency_from:str,currency_to:str,date:str='latest'):
    '''
    1. 取得目前查詢貨幣的匯率  
    2. 如果沒有指定日期,請設定currency_date=latest

	Args:  
        currency_date:如果沒有指定日期,請設定currency_date=latest,日期,格式必需是YYYY-MM-DD  
        currency_from:要被轉換的貨幣名稱,必需附合`ISO 4217`  
        currency_from:要轉換成為的貨敝名稱,必需附合`ISO 4217`

	Returns:  
        傳出一指定的dictionary,各個key的說明如下  
        date:查詢的日期  
        from_currency:當作基準貨幣名稱  
        to_currency:當作要轉換成為的貨幣名稱  
        rate:匯率
    '''
    if not date:
        date = 'latest' 
    url = f'https://api.frankfurter.app/{date}?base={currency_from}&symbols={currency_to}'
    response = requests.get(url)
    rate = response.json()['rates'][currency_to]
    return{
        "date":date,
        "from_currency":currency_from,
        "to_currency":currency_to,
        "rate":rate
    }

In [61]:
#手動測試function
get_exchange_rate('USD',"AUD")

{'date': 'latest', 'from_currency': 'USD', 'to_currency': 'AUD', 'rate': 1.607}

In [67]:
#由model chat來呼叫
import google.generativeai as genai
import os
from IPython.display import display,Markdown 

genai.configure(api_key=os.environ['GEMINI_API_KEY'])
model = genai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    tools=[get_exchange_rate],
    system_instruction='''
    如果沒有指定日期,請設定date='latest'
    '''
)

chat = model.start_chat(enable_automatic_function_calling=True)
response = chat.send_message('今日100美金對換澳幣是多少錢?')
print(response.text)
print(chat.last)


今日 100 美元可兌換 160.7 澳元。

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "\u4eca\u65e5 100 \u7f8e\u5143\u53ef\u514c\u63db 160.7 \u6fb3\u5143\u3002\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
    